<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2023noteooks/2023_0213RAM_check_pretrained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 訓練済モデルのチェック

In [1]:
import torch
from IPython import get_ipython
isColab =  'google.colab' in str(get_ipython())
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
%reload_ext autoreload
%autoreload 2

if isColab:

    # termcolor を downgrade しないと colab ではテキストに色がつかない
    !pip install --upgrade termcolor==1.1
    import termcolor    

    # colab 上で MeCab を動作させるために，C コンパイラを起動して，MeCab の構築を行う
    # そのため時間がかかる。
    !apt install aptitude
    !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
    !pip install mecab-python3==0.7
    !pip install jaconv
    !pip install japanize_matplotlib
    
    import MeCab
    mecab_wakati = MeCab.Tagger('-Owakati').parse
    mecab_yomi = MeCab.Tagger('-Oyomi').parse
else:
    from ccap.mecab_settings import yomi as mecab_yomi
    from ccap.mecab_settings import wakati as mecab_wakati

    import os
    if not os.path.exists('RAM'):
        !git clone https://github.com/ShinAsakawa/RAM.git 
    
    !pip install jaconv

from RAM.dataset import *
from RAM import check_vals_performance
from RAM import set_model_from_param_file
from RAM import check_fushimi1999_words

In [ ]:
fname = 'RAM/2023_0213ram_vdrj.pt'
fname = 'RAM/2023_0213ram.pt'
encoder, decoder, encoder_optimizer, decoder_optimizer, params, ds = set_model_from_param_file(
    fname=fname)
fushimi1999_dataset = Fushimi1999_Dataset(source=params['source'], target=params['target'])

check_vals_performance(
    encoder=encoder, decoder=decoder,
    _dataset={'Fushimi1999':fushimi1999_dataset},
    source_vocab=ds.source_list, target_vocab=ds.target_list,
    max_length=ds.maxlen, device=device)

In [ ]:
check_fushimi1999_words(encoder=encoder, decoder=decoder, cr_every=4, ds=ds)

In [ ]:
from RAM import eval_input_seq2seq
eval_input_seq2seq(encoder=encoder, decoder=decoder, ds=ds)